### 1.2. Minimization problem on distribution

Discrete approximation $F_{\Delta S, T}(x)$ of the continuous distribution 
$F_{dS}(x)$ can be estimated in two ways: theoretical and empirical. 

On one hand:
$$
F^{(theor)}_{\Delta S, T}(s) =
\sum_{k=1}^{K}{p_k \cdot \Phi \left( \frac{s - y_k \Delta t}{z_k \Delta t} \right)} 
\xrightarrow{K \rightarrow \infty , \,\Delta t \rightarrow 0} F_{dS}
$$

But on the other we can estimate empirical distribution of $\Delta S$ as:

$$
F^{(emp)}_{\Delta S, T}(s) = 
\frac{1}{T} \cdot \sum_{j=1}^{T} \mathbb{I} \left(\Delta S < s\right)
\xrightarrow{\Delta t \rightarrow 0} F_{dS}
$$

For evaluation of parameters $V_k = \{y_k, z_k, p_k\}$ we can set an 
minimization problem on distance (in some metric $\rho(f,g)$) between
theoretical and empirical distributions:


$$ 
\begin{equation}
\left\{ \begin{aligned} 

    \min_{\{V_k\}_{k=1}^{K}} \rho \left(F^{(emp)}_{\Delta S, T}, F^{(theor)}_{\Delta S, T}\right) \\
    p_k \geq 0 \; \wedge \; \sum_{k=1}^{K}p_k = 1 \\
    0 \leq z_k \in B \subset \mathbb{R^+} \\
    y_k \in A \subset \mathbb{R}



\end{aligned} \right.
\end{equation}
$$


## Neural network

We have a problem of approximation of known function - empirical distributon
Let's denote observed and calculated values as next:
$$
\mathbf{x} = \left[x_1, x_2, \dots, x_N\right] \text{ - input values,} \\
\mathbf{a} = \Delta t \cdot \left[y_1, y_2, \dots, y_K\right] \text{- expectation values,}\\
\mathbf{b} = \Delta t \cdot \left[z_1, z_2, \dots, z_K\right] \text{- standard deviation values,}\\
\mathbf{p} = \left[p_1, p_2, \dots, p_K\right] \text{- joint probability density values}\\
\mathbf{y} = F^{(emp)}_{\Delta S, T}(x) = \bigl\{F^{(emp)}_{\Delta S, T} = f\bigr\} = \left[f(x_1), f(x_2), \dots, f(x_M)\right] \text{- targets} \\
$$

Then we can write theoretical distribution in the next form:
$$
\^{y} = p \cdot \Phi{\left( \right)} \\ 
\^{y} = p_1 \cdot \Phi{\left( c_1\right)} + \dots + p_K \cdot \Phi{\left( c_K\right)} \\
c_n = 
$$

<img src="../src/docs/nn.svg" alt="nn" class="bg-primary" width="1400px">
\begin{bmatrix}
1 & 2 & 3\\
a & b & c
\end{bmatrix}



https://docs.pytorch.org/tutorials/intermediate/parametrizations.html
https://medium.com/@minh.hoque/demystifying-neural-network-normalization-techniques-4a21d35b14f8
https://towardsdatascience.com/deep-dive-into-softmax-regression-62deea103cb8/
https://www.geeksforgeeks.org/deep-learning/what-is-layer-normalization/
https://medium.com/@aya_hesham/function-approximation-with-deep-learning-a-practical-guide-with-code-examples-b817a59755af


```{python}
import torch
import torch.nn as nn

class ConstrainedLinear(nn.Module):
    def __init__(self, in_features, out_features, eps=1e-7):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.eps = eps
        # Define raw weights as a learnable parameter
        self.weight_raw = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        # Normalize the raw weights to sum to 1 along a specific dimension
        # The .clamp(min=self.eps) prevents division by zero if the sum is zero
        normalized_weight = self.weight_raw / self.weight_raw.sum(dim=1, keepdim=True).clamp(min=self.eps)
        return torch.nn.functional.linear(x, normalized_weight, self.bias)

# Example usage
model = ConstrainedLinear(in_features=10, out_features=5)
input_tensor = torch.randn(1, 10)
output = model(input_tensor)
```

In [ ]:
import tensorflow as tf

window_size = 100
K = 64
model = tf.keras.models.Sequential(
    [
        tf.keras.Input(shape=(1,)),
        #   tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(K, activation="relu"),
        tf.keras.layers.Dense(2 * K, activation="relu"),
        #   tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1),
    ]
)

model.summary()

In [ ]:
import numpy as np

x_train = np.linspace(0, 2 * np.pi, 100)
y_train = np.sin(x_train)
model.compile(optimizer="adam", loss="mse")
model.fit(x_train, y_train, epochs=1000)  # Train for a sufficient number of epochs

In [ ]:
x_test = np.linspace(0, 2 * np.pi, 250)
y_test = np.sin(x_test)
x = np.expand_dims(x_test, axis=1)

predictions = model.predict(x)

In [ ]:
import pandas as pd

df = pd.DataFrame({"x": x_test, "sin": y_test, "nn": predictions.flatten()})

In [ ]:
import plotly.express as px

px.line(df, y=["sin", "nn"])